<a href="https://colab.research.google.com/github/123hawaa/ML-Project/blob/main/Fake_news_predection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News
Build a system to identify unreliable news articles

train.csv: A full training dataset with the following attributes:

*  id: unique id for a news article
*  title: the title of a news article
*  author: author of the news article
*  text: the text of the article; could be incomplete
*  label: a label that marks the article as potentially unreliable
*  1: unreliable
*  0: reliable








# import Libaries

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

* re (regular expression) is very useful-for searching the text in documents
* corpus mean the body of particular text,the important content of the text
Stopwords mean those words which doesn’t add much value to paragraphs like(a,and ,so,where)
So we will remove all stopsword because aren’t add much value
* stemming mean remove the prefix and suffix of the word to return the root of word

* Ifidvectorrizer ==convert the text into feature vector (numbers)

In [2]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# DATA Pre-proscesing

In [4]:
df = pd.read_csv('/content/F_N_train.csv',error_bad_lines=False, engine="python")

<ipython-input-4-7e8c3bf38b95>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/F_N_train.csv',error_bad_lines=False, engine="python")
Skipping line 3320: unexpected end of data


In [5]:
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [6]:
df.shape

(3318, 5)

In [7]:
df.isnull().sum()

id          0
title      92
author    334
text        7
label       0
dtype: int64

In [8]:
#fill the missing value with empety string
df = df.fillna(' ')

In [9]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
#instead of using the title columns for the predection ,we can use the just 2 columns thtat can make our model to work faster
df['content']= df['title'] + [' '] + df['author']

In [11]:
df.content

0       House Dem Aide: We Didn’t Even See Comey’s Let...
1       FLYNN: Hillary Clinton, Big Woman on Campus - ...
2       Why the Truth Might Get You Fired Consortiumne...
3       15 Civilians Killed In Single US Airstrike Hav...
4       Iranian woman jailed for fictional unpublished...
                              ...                        
3313    Here’s How To Predict What the Dems’ Next Move...
3314    24-minute game-changing video: Electronic ‘vot...
3315    Who Paid Hillary Clinton $21.7 Million to Give...
3316    Saudi Arabia Warns of Economic Fallout if Cong...
3317    Report: Hamas Leader Heads to Egypt for Rare T...
Name: content, Length: 3318, dtype: object

In [12]:
df.head(2)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."


stemming is when you reduce the word to root word

In [13]:
port_stem = PorterStemmer()

In [14]:
def Stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word)for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [15]:
df['content'] = df['content'].apply(Stemming)

In [16]:
df.content

0       hous dem aid even see comey letter jason chaff...
1       flynn hillari clinton big woman campu breitbar...
2                  truth might get fire consortiumnew com
3       civilian kill singl us airstrik identifi jessi...
4       iranian woman jail fiction unpublish stori wom...
                              ...                        
3313               predict dem next move meme tfpp writer
3314    minut game chang video electron vote machin wi...
3315    paid hillari clinton million give speech henri...
3316    saudi arabia warn econom fallout congress pass...
3317    report hama leader head egypt rare talk breitb...
Name: content, Length: 3318, dtype: object

In [17]:
X = df['content'].values
Y = df['label'].values

In [18]:
# convert text into numrical
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)


In [19]:
  print(X)

  (0, 6736)	0.2795540267998541
  (0, 5761)	0.25387678595456054
  (0, 3854)	0.34986172068058996
  (0, 3736)	0.30186925331757525
  (0, 3340)	0.24254013633929544
  (0, 3049)	0.22524558247518187
  (0, 2145)	0.240112588153804
  (0, 1660)	0.2669867429496026
  (0, 1575)	0.34986172068058996
  (0, 1293)	0.2572388002821329
  (0, 1088)	0.36757431243697164
  (0, 134)	0.285805333688448
  (1, 7144)	0.295684739391666
  (1, 2968)	0.1888186370783737
  (1, 2398)	0.7192097696490712
  (1, 1562)	0.2720841813356294
  (1, 1225)	0.19358059551126294
  (1, 987)	0.3765003555411492
  (1, 837)	0.1527150553435894
  (1, 667)	0.2882320417325174
  (2, 6711)	0.41840198606817347
  (2, 4150)	0.4966568902479398
  (2, 2602)	0.3474882815597673
  (2, 2346)	0.37501512914374063
  (2, 1360)	0.46197716366677266
  :	:
  (3315, 2937)	0.3922674126944228
  (3315, 2626)	0.3727560333989166
  (3315, 1225)	0.2311432885987846
  (3316, 7231)	0.10570485105255584
  (3316, 7016)	0.28026978046296136
  (3316, 6560)	0.10436268582134793
  (3316,

# split to train and test

In [23]:
# split to train and test
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2,stratify= Y, random_state = 2)

# Training the model using Logistic Regression

In [24]:
model = LogisticRegression()

In [25]:
model.fit(x_train,y_train)

LogisticRegression()

# Evaluation
accuracy score

In [26]:
#accuracy score in traning data
x_train_pre = model.predict(x_train)
x_train_score = accuracy_score(x_train_pre,y_train)

In [27]:
print(x_train_score)

0.9713639788997739


In [30]:
#accuracy score in test data
x_test_pre = model.predict(x_test)
x_test_score = accuracy_score(x_test_pre,y_test)

In [31]:
print(x_test_score)

0.9352409638554217
